In [1]:
import nltk
import re
import string
import unicodedata
import pandas as pd
import numpy as np
import datetime as dt
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
import urllib.request
import urllib.parse
import urllib.error
from bs4 import BeautifulSoup
import ssl
import json
import ast
import os
from urllib.request import Request, urlopen

In [2]:
#filename = '/Users/michaellee/Desktop/python_lyrics_project/granata_lyrics/free_beer/loyal_to_the_leaves'
#lyrics_file = open(filename, 'r')
#lyrics_content = lyrics_file.read()
#lyrics_file.close()

In [3]:
lemmatizer = WordNetLemmatizer()

In [4]:
validated_food_file_cleaned = pd.read_csv('validated_food_file_cleaned.csv')
food_category_df = pd.read_csv('parsed_food_descr_food_category.csv')

In [5]:
stopword_list = stopwords.words('english')
stopword_list = [re.sub('\W','',word.lower()) for word in stopword_list]

In [6]:
def strip_accents(text):

    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass

    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")

    return str(text)

In [7]:
# For ignoring SSL certificate errors

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [9]:
artist_agg_stats_df = pd.DataFrame(columns=['artist',
                                           'lyrics_word_count',
                                           'lyrics_word_count_excl_sw',
                                           'food_words_count',
                                           'food_words_perc_all_lyric_words',
                                           'food_words_perc_lyric_words_excl_sw',
                                           'song',
                                           'release_date',
                                           'release_year',
                                           'release_month'])
artist_food_word_stats_df = pd.DataFrame(columns=['food_word',
                                                  'food_word_count',
                                                  'artist',
                                                 'food_word_value',
                                                 'food_word_total',
                                                 'song',
                                                 'release_date',
                                                 'release_year',
                                                 'release_month',
                                                 'food_category_description'])

In [10]:
project_directory = '/Users/michaellee/Desktop/python_lyrics_project'
genius_domain = 'https://genius.com'

In [11]:
artist_page_urls_filename = '{0}/genius_artist_page_urls.txt'.format(project_directory)
artist_page_urls_file = open(artist_page_urls_filename, 'r')
artist_page_urls_list = artist_page_urls_file.readlines()
artist_page_urls_file.close()
artist_page_urls_list = [url.strip() for url in artist_page_urls_list]

In [12]:
artist_page_urls_list

['https://genius.com/artists/Mike-jones']

In [14]:
for artist_page_url in artist_page_urls_list:
    # Making the website believe that you are accessing it using a mozilla browser
    artist_page_url_req = Request(artist_page_url, headers = { 'User-Agent' : 'Mozilla/5.0' })
    artist_page_url_webpage = urlopen(artist_page_url_req).read()
    artist_page_soup = BeautifulSoup(artist_page_url_webpage, 'html.parser')
    artist_page_html = artist_page_soup.prettify('utf-8')
    
    artist_song_list_sub_url = [a['href'] for a in artist_page_soup.find_all("a",{"class":"full_width_button"}) if 'artists/songs' in a['href']][0]
    artist_song_list_fullpath_url = genius_domain + artist_song_list_sub_url
    print('Artist song page is: {0}'.format(artist_song_list_fullpath_url))
    
    # Making the website believe that you are accessing it using a mozilla browser
    artist_song_list_fullpath_url_req = Request(artist_song_list_fullpath_url, headers = { 'User-Agent' : 'Mozilla/5.0' })
    artist_song_list_fullpath_url_webpage = urlopen(artist_song_list_fullpath_url_req).read()
    
    # Creating a BeautifulSoup object of the html page for easy extraction of data.

    artist_song_list_page_soup = BeautifulSoup(artist_song_list_fullpath_url_webpage, 'html.parser')
    artist_song_list_page_html = artist_song_list_page_soup.prettify('utf-8')
    
    artist_song_lyrics_urls_list = []
    artist_song_list_page_pagination_range = []
    
    # FIND OUT THE NUMBER OF PAGES ON THE ARTIST SONG LIST PAGE "PAGINATION" TAG
    
    for tag in artist_song_list_page_soup.find_all("div",{"class":"pagination"}):
        for a in tag.find_all("a"):
            artist_song_list_page_pagination_range.append(int(a['href'].split('&')[2].split('=')[1]))
    
    try:
        artist_song_list_max_pages = max(artist_song_list_page_pagination_range)
    except:
        artist_song_list_max_pages = 1
    
    print('Artist has {0} pages of songs.'.format(artist_song_list_max_pages))
    
    for artist_song_list_page_number in range(1,artist_song_list_max_pages+1):
        print('Collecting lyrics URLs from page {0}'.format(artist_song_list_page_number))
        artist_song_list_page_num_url = artist_song_list_fullpath_url + '&page={0}&pagination=true'.format(artist_song_list_page_number)
        
        artist_song_list_page_num_url_req = Request(artist_song_list_page_num_url, headers = { 'User-Agent' : 'Mozilla/5.0' })
        artist_song_list_page_num_url_webpage = urlopen(artist_song_list_page_num_url_req).read()
        
        artist_song_list_page_num_soup = BeautifulSoup(artist_song_list_page_num_url_webpage, 'html.parser')
        artist_song_list_page_num_html = artist_song_list_page_num_soup.prettify('utf-8')
        
        for artist_song_lyric_url_wip in [a['href'] for a in artist_song_list_page_num_soup.find_all("a",{"class":"song_name work_in_progress song_link"})]:
            print('Adding {0} to the list of lyrics urls to be analyzed.'.format(artist_song_lyric_url_wip))
            artist_song_lyrics_urls_list.append(artist_song_lyric_url_wip)
        for artist_song_lyric_url_published in [a['href'] for a in artist_song_list_page_num_soup.find_all("a",{"class":"song_name published song_link"})]:
            print('Adding {0} to the list of lyrics urls to be analyzed.'.format(artist_song_lyric_url_published))
            artist_song_lyrics_urls_list.append(artist_song_lyric_url_published)
        for artist_song_lyric_url_under_review in [a['href'] for a in artist_song_list_page_num_soup.find_all("a",{"class":"song_name under_review song_link"})]:
            print('Adding {0} to the list of lyrics urls to be analyzed.'.format(artist_song_lyric_url_under_review))
            artist_song_lyrics_urls_list.append(artist_song_lyric_url_under_review)
    
    print('Collected all lyrics URLs. There are {0} in total. Time to analyze them for food words'.format(len(artist_song_lyrics_urls_list)))
    
    for lyrics_url in artist_song_lyrics_urls_list:
        url = lyrics_url
        analyzed_urls_filename = '{0}/analyzed_song_urls.txt'.format(project_directory)
        analyzed_urls_file = open(analyzed_urls_filename, 'r')
        analyzed_urls_file_list = analyzed_urls_file.readlines()
        analyzed_urls_file.close()

        if url.strip() in [analyzed_url.strip() for analyzed_url in analyzed_urls_file_list]:
            continue
        
        analyzed_urls_file = open(analyzed_urls_filename, 'a')
        analyzed_urls_file.write('{0}{1}'.format(url.strip(),'\n'))
        analyzed_urls_file.close()
        
        # Making the website believe that you are accessing it using a mozilla browser
        req = Request(url, headers = { 'User-Agent' : 'Mozilla/5.0' })
        webpage = urlopen(req).read()

        # Creating a BeautifulSoup object of the html page for easy extraction of data.

        soup = BeautifulSoup(webpage, 'html.parser')
        html = soup.prettify('utf-8')
        song_json = {}
        song_json["Lyrics"] = [];

        #Extract Title of the song
        for title in soup.findAll('title'):
            song_json["Title"] = title.text.strip()
        
        if '[Artwork]' in song_json["Title"]:
            continue
        
        try:
            song = song_json["Title"].split('|')[0].split('–')[1].lower().replace('lyrics','').strip()
        except:
            try:
                song = song_json["Title"].split('|')[0].split('-')[1].lower().replace('lyrics','').strip()
            except:
                continue

        # Extract the release date of the song
        if len(soup.findAll('span', attrs = {'class': 'metadata_unit-info metadata_unit-info--text_only'})) == 0:
            release_date_formatted = 0
            release_year = 0
            release_month = 0
        else:
            for span in soup.findAll('span', attrs = {'class': 'metadata_unit-info metadata_unit-info--text_only'}):
                song_json["Release date"] = span.text.strip()

            release_date = song_json["Release date"]
            if len(release_date) == 0:
                release_date_formatted = 0
                release_year = 0
                release_month = 0
            else:
                try:
                    release_date_formatted = dt.datetime.strptime(release_date, '%B %d, %Y').strftime('%Y-%m-%d')
                    release_year = dt.datetime.strptime(release_date, '%B %d, %Y').year
                    release_month = dt.datetime.strptime(release_date, '%B %d, %Y').month
                except:
                    try:
                        release_date_formatted = dt.datetime.strptime(release_date, '%Y').strftime('%Y-%m-%d')
                        release_year = dt.datetime.strptime(release_date, '%Y').year
                        release_month = dt.datetime.strptime(release_date, '%Y').month
                    except:
                        release_date_formatted = 0
                        release_year = 0
                        release_month = 0

        #Extract the Lyrics of the song
        for div in soup.findAll('div', attrs = {'class': 'lyrics'}):
            song_json["Lyrics"].append(div.text.strip().split("\n"));

        artist_lyric_words = {}

        for line in song_json["Lyrics"][0]:
            if len(line) == 0:
                verse_artists_unique = []
                continue
            if line[0] == '[':
                verse_artists_unique = []
                line = line.strip('[]')
                line = line.split(':')
                if len(line) == 1:
                    verse_artists = song_json["Title"].split('|')[0].split('–')[0].lower().strip()
                    verse_artists = verse_artists.replace('tyler, the creator','tyler the creator')
                    verse_artists_unique.append(verse_artists)
                    continue
                else:
                    verse_artists_unique = []
                    verse_artists = line[1]
                    verse_artists = verse_artists.lower()
                    verse_artists = strip_accents(verse_artists)
                    verse_artists = verse_artists.replace('both','')
                    verse_artists = verse_artists.replace('tyler, the creator','tyler the creator')
                    verse_artists = verse_artists.split(',')
                    for artists in verse_artists:
                        artists = artists.replace('(','')
                        artists = artists.replace(')','')
                        if '&' in artists:
                            artists = artists.split('&')
                        if '+' in artists:
                            artists = artists.split('+')
                        if type(artists) == str:
                            artists = artists.strip()
                            verse_artists_unique.append(artists)
                        if type(artists) == list:
                            for artist in artists:
                                artist = artist.strip()
                                verse_artists_unique.append(artist)
                    verse_artists_unique = list(set(verse_artists_unique))
                    continue
                    
            words = line.split()
            for word in words:
                word = re.sub('\W','',word.lower())
                word = lemmatizer.lemmatize(word)
                word = strip_accents(word)
                if len(verse_artists_unique) == 0:
                    verse_artist = song_json["Title"].split('|')[0].split('–')[0].lower().strip()
                    verse_artists_unique.append(verse_artist)
                for artist in verse_artists_unique:
                    if artist in artist_lyric_words:
                        artist_lyric_words[artist].append(word)
                    else:
                        artist_lyric_words[artist] = []
                        artist_lyric_words[artist].append(word)

        artist_lyric_words_count = {}
        artist_lyric_words_count_excl_stopwords = {}
        lyric_words_count_total = []
        lyric_words_excl_stopwords_count_total = []

        for artist in artist_lyric_words.keys():
            artist_lyric_words_count[artist] = len(artist_lyric_words[artist])
            artist_lyric_words_count_excl_stopwords[artist] = len([word for word in artist_lyric_words[artist] if word not in stopword_list])
        
        for artist in artist_lyric_words_count.keys():
            lyric_words_count_total.append(artist_lyric_words_count[artist])
            
        for artist in artist_lyric_words_count_excl_stopwords.keys():
            lyric_words_excl_stopwords_count_total.append(artist_lyric_words_count_excl_stopwords[artist])
            
        if sum(lyric_words_count_total) == 0:
            continue
        if sum(lyric_words_excl_stopwords_count_total) == 0:
            continue
            
        artist_lyrics_content_one_word_dict = {}
        artist_lyrics_content_two_word_dict = {}

        for artist in artist_lyric_words.keys(): 
            if len(artist_lyric_words[artist]) == 0:
                continue
            for word_num in range(artist_lyric_words_count[artist]-1):
                if word_num < artist_lyric_words_count[artist]:
                    one_word = artist_lyric_words[artist][word_num]
                    if artist in artist_lyrics_content_one_word_dict:
                        if one_word in artist_lyrics_content_one_word_dict[artist]:
                            artist_lyrics_content_one_word_dict[artist][one_word] += 1
                        else:
                            artist_lyrics_content_one_word_dict[artist][one_word] = 1
                    else:
                        artist_lyrics_content_one_word_dict[artist] = {}
                        artist_lyrics_content_one_word_dict[artist][one_word] = 1

                    two_word = artist_lyric_words[artist][word_num] + ' ' + artist_lyric_words[artist][word_num+1]
                    if artist in artist_lyrics_content_two_word_dict:
                        if two_word in artist_lyrics_content_two_word_dict[artist]:
                            artist_lyrics_content_two_word_dict[artist][two_word] += 1
                        else:
                            artist_lyrics_content_two_word_dict[artist][two_word] = 1
                    else:
                        artist_lyrics_content_two_word_dict[artist] = {}
                        artist_lyrics_content_two_word_dict[artist][two_word] = 1    

        artist_lyric_word_count_df = pd.DataFrame(columns=['word',
                                                      'word_count',
                                                      'artist'])

        for artist in artist_lyrics_content_one_word_dict.keys():
            artist_lyrics_content_one_word_df = pd.DataFrame.from_dict(artist_lyrics_content_one_word_dict[artist], orient='index')
            artist_lyrics_content_one_word_df = artist_lyrics_content_one_word_df.reset_index()
            artist_lyrics_content_one_word_df['artist'] = artist
            artist_lyrics_content_one_word_df.columns = ['word','word_count','artist']
            artist_lyric_word_count_df = pd.concat([artist_lyric_word_count_df,artist_lyrics_content_one_word_df],axis=0,ignore_index=True)

        for artist in artist_lyrics_content_two_word_dict.keys():
            artist_lyrics_content_two_word_df = pd.DataFrame.from_dict(artist_lyrics_content_two_word_dict[artist], orient='index')
            artist_lyrics_content_two_word_df = artist_lyrics_content_two_word_df.reset_index()
            artist_lyrics_content_two_word_df['artist'] = artist
            artist_lyrics_content_two_word_df.columns = ['word','word_count','artist']
            artist_lyric_word_count_df = pd.concat([artist_lyric_word_count_df,artist_lyrics_content_two_word_df],axis=0,ignore_index=True)


        artist_food_words_df = pd.merge(artist_lyric_word_count_df,validated_food_file_cleaned[['food_description','food_word_net_effect_value']],how='inner',left_on='word',right_on='food_description')
        artist_food_words_df = artist_food_words_df.drop(labels='food_description',axis=1)
        artist_food_words_df = artist_food_words_df.rename(columns={'food_word_net_effect_value':'food_word_value'})
        artist_food_words_df['food_word_total'] = artist_food_words_df['word_count'] * artist_food_words_df['food_word_value']
        artist_food_words_df = artist_food_words_df.rename(columns={'word':'food_word',
                                                                   'word_count':'food_word_count'})
        artist_food_words_df['song'] = song
        artist_food_words_df['release_date'] = release_date_formatted
        artist_food_words_df['release_year'] = release_year
        artist_food_words_df['release_month'] = release_month
        artist_food_words_df = pd.merge(artist_food_words_df,food_category_df,how='left',left_on='food_word',right_on='food_description')
        artist_food_words_df = artist_food_words_df.drop(labels=['food_description','food_category_id'],axis=1)

        artist_food_words_count = {}
        artist_food_words_perc_all_words = {}
        artist_food_words_perc_all_words_excl_sw = {}

        for artist in list(artist_lyric_words_count.keys()):
            artist_food_words_count[artist] = sum(artist_food_words_df.loc[artist_food_words_df.artist == artist]['food_word_total'])
            try:
                artist_food_words_perc_all_words[artist] = artist_food_words_count[artist] / artist_lyric_words_count[artist]
            except:
                artist_food_words_perc_all_words[artist] = 0
            try:
                artist_food_words_perc_all_words_excl_sw[artist] = artist_food_words_count[artist] / artist_lyric_words_count_excl_stopwords[artist]
            except:
                artist_food_words_perc_all_words_excl_sw[artist] = 0
                
        artist_lyric_words_df = pd.DataFrame.from_dict(artist_lyric_words_count,orient='index')
        artist_lyric_words_df.columns = ['lyrics_word_count']
        artist_lyric_words_excl_sw_df = pd.DataFrame.from_dict(artist_lyric_words_count_excl_stopwords,orient='index')
        artist_lyric_words_excl_sw_df.columns = ['lyrics_word_count_excl_sw']
        artist_food_words_count_df = pd.DataFrame.from_dict(artist_food_words_count,orient='index')
        artist_food_words_count_df.columns = ['food_words_count']
        artist_food_words_perc_all_words_df = pd.DataFrame.from_dict(artist_food_words_perc_all_words,orient='index')
        artist_food_words_perc_all_words_df.columns = ['food_words_perc_all_lyric_words']
        artist_food_words_perc_all_words_excl_sw_df = pd.DataFrame.from_dict(artist_food_words_perc_all_words_excl_sw,orient='index')
        artist_food_words_perc_all_words_excl_sw_df.columns = ['food_words_perc_lyric_words_excl_sw']

        song_agg_stats = pd.concat([artist_lyric_words_df,
                                    artist_lyric_words_excl_sw_df,
                                    artist_food_words_count_df,
                                    artist_food_words_perc_all_words_df,
                                   artist_food_words_perc_all_words_excl_sw_df],axis=1)
        song_agg_stats = song_agg_stats.replace(np.nan,0)
        song_agg_stats = song_agg_stats.reset_index()
        song_agg_stats = song_agg_stats.rename(columns={'index':'artist'})
        song_agg_stats['song'] = song
        song_agg_stats['release_date'] = release_date_formatted
        song_agg_stats['release_year'] = release_year
        song_agg_stats['release_month'] = release_month

        artist_agg_stats_df = pd.concat([artist_agg_stats_df,song_agg_stats],axis=0,ignore_index=True)
        artist_food_word_stats_df = pd.concat([artist_food_word_stats_df,artist_food_words_df],axis=0,ignore_index=True)
        print('Analyzed {0} for food words'.format(lyrics_url))


Artist song page is: https://genius.com/artists/songs?for_artist_page=287&id=Mike-jones
Artist has 12 pages of songs.
Adding https://genius.com/Mike-jones-21-questions-lyrics to the list of lyrics urls to be analyzed.
Adding https://genius.com/Mike-jones-21-questions-freestyle-lyrics to the list of lyrics urls to be analyzed.
Adding https://genius.com/Roy-jones-jr-24s-lyrics to the list of lyrics urls to be analyzed.
Adding https://genius.com/Mike-jones-3-grams-lyrics to the list of lyrics urls to be analyzed.
Adding https://genius.com/Mike-jones-5-years-from-now-lyrics to the list of lyrics urls to be analyzed.
Adding https://genius.com/Mike-jones-5-years-from-now-remix-lyrics to the list of lyrics urls to be analyzed.
Adding https://genius.com/Mike-jones-5-years-from-now-screwed-and-chopped-feat-lil-bran-lyrics to the list of lyrics urls to be analyzed.
Adding https://genius.com/Mike-jones-8-mile-lyrics to the list of lyrics urls to be analyzed.
Adding https://genius.com/Mike-jones-a

Adding https://genius.com/Smug-mang-gualla-lyrics to the list of lyrics urls to be analyzed.
Adding https://genius.com/Mike-jones-hallelujah-lyrics to the list of lyrics urls to be analyzed.
Adding https://genius.com/Mike-jones-happy-birthday-lyrics to the list of lyrics urls to be analyzed.
Adding https://genius.com/Mike-jones-hate-on-me-lyrics to the list of lyrics urls to be analyzed.
Adding https://genius.com/Mike-jones-hey-ma-lyrics to the list of lyrics urls to be analyzed.
Adding https://genius.com/Emune-hittin-switches-lyrics to the list of lyrics urls to be analyzed.
Adding https://genius.com/Bun-b-hold-u-down-lyrics to the list of lyrics urls to be analyzed.
Adding https://genius.com/The-wreckshop-family-holl-up-lyrics to the list of lyrics urls to be analyzed.
Adding https://genius.com/Mike-jones-houston-oilers-lyrics to the list of lyrics urls to be analyzed.
Adding https://genius.com/Justin-hibbert-i-i-am-work-feat-mike-jones-lyrics to the list of lyrics urls to be analyze

Adding https://genius.com/Mike-jones-rookie-of-da-year-lyrics to the list of lyrics urls to be analyzed.
Adding https://genius.com/La-chat-round-and-round-lyrics to the list of lyrics urls to be analyzed.
Adding https://genius.com/Swishahouse-rulez-lyrics to the list of lyrics urls to be analyzed.
Adding https://genius.com/Mike-jones-scandalous-hoes-lyrics to the list of lyrics urls to be analyzed.
Adding https://genius.com/Mike-jones-scandalous-hoes-ii-lyrics to the list of lyrics urls to be analyzed.
Adding https://genius.com/Mike-jones-scandalous-hoes-remix-lyrics to the list of lyrics urls to be analyzed.
Adding https://genius.com/Mike-jones-scandalous-hoes-screwed-and-chopped-feat-lil-bran-lyrics to the list of lyrics urls to be analyzed.
Adding https://genius.com/Mike-jones-scandalous-hoes-skit-lyrics to the list of lyrics urls to be analyzed.
Adding https://genius.com/Mike-jones-screw-dat-lyrics to the list of lyrics urls to be analyzed.
Adding https://genius.com/Mike-jones-scre

Analyzed https://genius.com/Mike-jones-back-then-re-recorded-lyrics for food words
Analyzed https://genius.com/Mike-jones-back-then-screwed-and-chopped-lyrics for food words
Analyzed https://genius.com/Ying-yang-twins-badd-lyrics for food words
Analyzed https://genius.com/Mike-jones-boi-lyrics for food words
Analyzed https://genius.com/Mike-jones-boi-young-problemz-and-mike-jones-lyrics for food words
Analyzed https://genius.com/Bizzy-bone-brain-on-drugs-lyrics for food words
Analyzed https://genius.com/Mike-jones-call-me-when-you-need-me-lyrics for food words
Analyzed https://genius.com/Mike-jones-cash-that-i-got-lyrics for food words
Analyzed https://genius.com/Swishahouse-cash-that-i-got-lyrics for food words
Analyzed https://genius.com/Mike-jones-champagne-music-lyrics for food words
Analyzed https://genius.com/Mike-jones-crunch-n-munch-lyrics for food words
Analyzed https://genius.com/Cinn-tree-crew-ct-rap-lyrics for food words
Analyzed https://genius.com/Mike-jones-cuddy-buddy-al

Analyzed https://genius.com/Mike-jones-perfect-team-pt-3-lyrics for food words
Analyzed https://genius.com/Daye-lahookz-pop-dat-the-remix-lyrics for food words
Analyzed https://genius.com/Mike-jones-quick-2-back-down-lyrics for food words
Analyzed https://genius.com/Mike-jones-quick-to-back-down-flow-lyrics for food words
Analyzed https://genius.com/Hakeem-ture-remember-what-you-sign-lyrics for food words
Analyzed https://genius.com/Mike-jones-riot-lyrics for food words
Analyzed https://genius.com/Mike-jones-next-to-you-lyrics for food words
Analyzed https://genius.com/Mike-jones-rookie-of-da-year-lyrics for food words
Analyzed https://genius.com/La-chat-round-and-round-lyrics for food words
Analyzed https://genius.com/Swishahouse-rulez-lyrics for food words
Analyzed https://genius.com/Mike-jones-scandalous-hoes-lyrics for food words
Analyzed https://genius.com/Mike-jones-scandalous-hoes-remix-lyrics for food words
Analyzed https://genius.com/Mike-jones-scandalous-hoes-screwed-and-chop

In [15]:
artist_food_word_stats_df[:50]

,food_word,food_word_count,artist,food_word_value,food_word_total,song,release_date,release_year,release_month,food_category_description
0,food,1,mike jones,1,1,5 years from now,2005-04-19,2005,4,NaN
1,food,1,mike jones,1,1,5 years from now (remix),0,0,0,NaN
2,food,1,mike jones,1,1,5 years from now (screwed & chopped) (feat. li...,0,0,0,NaN
3,baguette,1,mike jones,1,1,addictive,0,0,0,NaN
4,candy,2,mike jones,1,2,addictive,0,0,0,Sweets
5,butter,1,mike jones,1,1,addictive,0,0,0,Dairy and Egg Products
6,gin,1,mike jones,1,1,addictive,0,0,0,NaN
7,bacardi,1,mike jones,1,1,addictive,0,0,0,NaN
8,bread,1,mike jones,1,1,addictive,0,0,0,Baked Products
9,cake,1,mike jones,1,1,adonis blue,1991-08-12,1991,8,Baked Products


In [16]:
artist_food_word_stats_df.to_csv('artist_food_word_stats24.csv',index=False)

In [17]:
artist_agg_stats_df.sort_values(by='food_words_perc_lyric_words_excl_sw',ascending=False)

,artist,lyrics_word_count,lyrics_word_count_excl_sw,food_words_count,food_words_perc_all_lyric_words,food_words_perc_lyric_words_excl_sw,song,release_date,release_year,release_month
168,mike jones,207,135,22,0.106280,0.162963,leanin on dat butter,0,0,0
72,rnb base,174,81,12,0.068966,0.148148,drinks on me,2016-01-01,2016,1
123,mike jones,12,9,1,0.083333,0.111111,fresh azimiz (remix),0,0,0
101,mike jones,20,9,1,0.050000,0.111111,gangsta grillz,2006-07-25,2006,7
205,,31,18,2,0.064516,0.111111,on top of the covers,0,0,0
...,...,...,...,...,...,...,...,...,...,...
1,mike jones,326,149,0,0.000000,0.000000,21 questions freestyle,0,0,0
167,lil' bran,2,1,0,0.000000,0.000000,leanin on dat butter,0,0,0
170,mike jones,272,119,0,0.000000,0.000000,let me show you,0,0,0
171,gary stockdale,17,11,0,0.000000,0.000000,liftoff of love / ripoff of love,2002-01-01,2002,1


In [18]:
artist_agg_stats_df.to_csv('artist_agg_stats24.csv',index=False)